# Association or Market basket analysis - online transactions

## Introduction

Association analysis (most commonly known as "Market Basket Analysis") is a type of unsupervised analysis that seeks to find hidden patterns or connections in data through identifying instances of co-occurrence. The meaning of co-ocurrence can be explained in regards to customer shopping behavior where we ask the question "given the occurrence of product A (the antecedent) in a transaction, how likely are we to find the occurrence of product B (the consequent)?"(Provost & Fawcett,2013).  An example could be the co-occurrence of cereal and milk in a hypothetical grocery store transaction. This can also include groups of more than one item (e.g., the occurrence of peanut butter and jelly in a transaction, may lead to the purchase of bread (Lantz,015)). 

For this notebook, I will be exploring 'Association' analysis in regards to online customer transactions, to learn patterns in the purchasing data. This knowledge from mining patterns in customer shopping behavior can then be leveraged in many ways: to make better product recommendations, create cross-selling opportunities (placing items next to each other in an aisle), better inform marketing decisions, and even inform the logistics of warehousing inventory.<br>

Sources:<br>
Lantz, B. (2015). Machine Learning with R - Second Edition. Birmingham, UK: Packt Publishing.<br>

Provost, F., &amp; Fawcett, T. (2013). Data science for business: What you need to know about data mining and data-analytic thinking. Sebastopol, CA: O'Reilly.<br>

### Changes
List dates and brief description

12-20-2020: notebook initiated

### Data Sources/Related Files
The data for this project is to explore a dataset of customer transactions that occurred between 01/12/2010 and 09/12/2011 from a UK based online retail company. This dataset can be found at http://archive.ics.uci.edu/ml/datasets/Online+Retail and was donated to the UCI Machine Learning Repository by Dr Daqing Chen.

The data consists of 541909 records and 8 attributes:<br>

<b>InvoiceNo</b>: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.<br>
<b>StockCode</b>: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.<br>
<b>Description</b>: Product (item) name. Nominal.<br>
<b>Quantity</b>: The quantities of each product (item) per transaction. Numeric.<br>
<b>InvoiceDate</b>: Invice Date and time. Numeric, the day and time when each transaction was generated.<br>
<b>UnitPrice</b>: Unit price. Numeric, Product price per unit in sterling.<br>
<b>CustomerID</b>: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.<br>
<b>Country</b>: Country name. Nominal, the name of the country where each customer resides.<br>

### Imports
Import libraries and write settings here.

In [1]:
# Data manipulation
import datetime
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import numpy as np


# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50
pd.options.display.width = 100

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
sns.set()

# Data Processing
Clean up columns and datatypes to prepare for analysis

In [2]:
# read in the data
file = "C:/Users/Jeff/data/Online Retail.csv"
df = pd.read_csv(file)
#inspect dtypes and counts
df.info()
#quick sample of the data
df.sample(n = 15, random_state = 42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
209268,555200,71459,HANGING JAM JAR T-LIGHT HOLDER,24,6/1/2011 12:05,0.85,17315.0,United Kingdom
207108,554974,21128,GOLD FISHING GNOME,4,5/27/2011 17:14,6.95,14031.0,United Kingdom
167085,550972,21086,SET/6 RED SPOTTY PAPER CUPS,4,4/21/2011 17:05,0.65,14031.0,United Kingdom
471836,576652,22812,PACK 3 BOXES CHRISTMAS PANETTONE,3,11/16/2011 10:39,1.95,17198.0,United Kingdom
115865,546157,22180,RETROSPOT LAMP,2,3/10/2011 8:40,9.95,13502.0,United Kingdom
465024,576200,82482,WOODEN PICTURE FRAME WHITE FINISH,2,11/14/2011 12:14,2.95,15572.0,United Kingdom
477777,577076,22614,PACK OF 12 SPACEBOY TISSUES,12,11/17/2011 15:08,0.39,14362.0,United Kingdom
367855,568909,22596,CHRISTMAS STAR WISH LIST CHALKBOARD,12,9/29/2011 13:38,1.25,16818.0,United Kingdom
491657,578072,21109,LARGE CAKE TOWEL CHOCOLATE SPOTS,1,11/22/2011 16:02,6.75,17759.0,United Kingdom
269641,560491,23297,SET 40 HEART SHAPE PETIT FOUR CASES,2,7/19/2011 10:51,1.65,12415.0,Australia


I had quickly profiled the data ahead of loading the data in this nootbook to check for anomalies or other issues using the Excel filtering feature. A few things that stood out from that quik scan that need to be quickly taken care of are:

1. Null values for product descriptions in 'Description' attribute (1454 obs)
2. Whitespace issues in the product descriptions (4223 obs originally, 4211 obs after cleanup)
3. Some 'InvoiceNo' values contain a 'C', which appear to indicate cancelations, discounts, and returns and so are not real purchases (9288 obs)
4. There are records where the 'Quantity' is negative. This appears to also indicate fees or returns. I worry keeping these in will be false associations and therefore may influence the association algorithm (10624 obs)

In [3]:
# checking for null values in the data
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

There are nulls found in the 'Description' and 'CustomerID' attributes. Since I am unaware of what the products are without a description and have no way to fill them, I will remove them for simplicity's sake. The nulls for 'CustomerID' will not affect this analysis and so we will do nothing there.

In [4]:
# drop records where product description is null
df = df.dropna(subset = ['Description'])

In [5]:
# remove white space from the description text
df['Description'] = df['Description'].str.strip()

In [6]:
# confirm count of all unique products
all_products = df['Description'].unique()
print("Total products: {}".format(len(all_products)))

Total products: 4211


In [7]:
# drop all rows with negative 'Quantity' - this should also remove all 'TransactionID' instances containing "C"
before = len(df)
print("Before dropping negative Quantity: " + str(before))
df = df.loc[df['Quantity']> 0]
after = len(df)
print("After dropping negative Quantity: " + str(after))

Before dropping negative Quantity: 540455
After dropping negative Quantity: 530693


In [8]:
# double check that no 'TransactionID' contain "C"
has_C = df['InvoiceNo'].str.contains("C").sum()

print("Count of InvoiceNo that contain \'C\': {}" .format(has_C))

Count of InvoiceNo that contain 'C': 0


<b>The following piece of code below comes from the "Practical Business Python" blog https://pbpython.com/market-basket-analysis.html.
I appreciate how simple and clean the approach is to convert the dataset to one-hot encoding for Association Analysis via groupby and a helper function.</b>

In [9]:
# lets just work only with the UK, grouped by invoice and description, unstacked and fill all null values with 0
sales_UK = (df[df['Country'] == "United Kingdom"].groupby(['InvoiceNo', 'Description'])['Quantity'].sum()
    .unstack().reset_index().fillna(0)
    .set_index('InvoiceNo')
           )

# next we convert all counts to 0 or 1 using a helper function
def encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

sales = sales_UK.applymap(encode)

In [10]:
sales_UK.shape

(18194, 4046)

(18194, 4046)

# Analysis/Modeling


The transformed data now consists of 18,194 unique transactions and 4046 unique products. I will now begin working with the mlxtend library for "apriori" and "association rules" to identify any unique product associations in the data. First I need to establish my floor support on interesting transactions.<br><br>
<b>Support</b> refers to how frequently an itemset is occuring in the data. I am using a minimum support here of 0.03, which seems sensible given the amount of transactions we are analyzing. 

In [11]:
frequent_itemsets = apriori(sales, min_support=0.03, use_colnames=True)
print("{} itemsets have been identified".format(len(frequent_itemsets)))

140 itemsets have been identified


Next, I am going to call the association rules to find our interesting rules in the data. In the arguments I am using the <b>lift</b> metric which is a ratio of the observed support where the consequent will happen given the antecedent, divided by the probability that they both occur by chance.<br>
- When **lift equals 1**, then the antecedent and consequent are independent of eachother and do not have a relationship. 
- When **lift \< 1**, then the less chance there is that the occurence of the consequent will happen given the antecedent meaning that their relationship is negative
- When **lift > 1**, then the more likely that the occurence of the consequent will happen given the antecedent and so we have a positive relationship

So in this case, I will be wanting to find all rules where the lift is greater than 1 which indicates a poitive relationship.

In [12]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print ("{} rules have been identified".format(len(rules)))

20 rules have been identified


# Results


Below are the resulting rules that were identified per my parameters of a "Support" of 0.03 and a "Lift" minimum of 1. I am sorting the results by strength of the rules' "Confidence"

In [13]:
# display the rules and sort them by confidence greatest to least
rules.sort_values(by = "confidence", ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.038639,0.051336,0.031714,0.820768,15.988282,0.029730,5.292945
19,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.038639,0.052600,0.030010,0.776671,14.765684,0.027977,4.242181
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.051336,0.052600,0.038529,0.750535,14.268798,0.035829,3.797733
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.052600,0.051336,0.038529,0.732497,14.268798,0.035829,3.546374
9,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.063702,0.106519,0.043146,0.677308,6.358587,0.036361,2.768837
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.048148,0.051116,0.030944,0.642694,12.573307,0.028483,2.655663
7,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.050016,0.106519,0.031329,0.626374,5.880414,0.026001,2.391377
2,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.051336,0.038639,0.031714,0.617773,15.988282,0.029730,2.515157
12,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.062108,0.106519,0.038364,0.617699,5.798977,0.031749,2.337116
0,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.051116,0.048148,0.030944,0.605376,12.573307,0.028483,2.412051


# Conclusions


Reviewing the list, the first four rules deal with dfferent color options of the "REGENCY TEACUP AND SAUCER" products. this implies that there is a strong connection where if a customer purchases a style of these teacups, then they will also buy another style of the same teacup and saucer.<br>

The strongest rule, with a support of over 0.03, confidence of 0.82 and lift of 15.98 is the "PINK REGENCY TEACUP AND SAUCER", where if it is purchased then it is highly likely that a customer will also buy the "GREEN REGENCY TEACUP AND SAUCER".<br>

Other items in the list of rules are also similar to the tea cups in that the consequent of a particular rule is a variant style of the atecedent such as alarm clocks of different colors and different styles of bags. These rules make sense to me as the data set represents wholesale customers of this particular online retailer, where the customers will generally need to buy a type of product in a selection of colors or styles for the retail customers.

If I was more familiar with the data, I may have other specific ideas/questions that I would like to delve deeper into.<br>
If I were to pursue this analysis further, I would also consider running this for each of the countries separately and then comparing the rules to see if there are similar patterns.
   